# 训练

In [1]:
with open("scp.1-cpu版.py", "r") as f:
    js = f.read()
    # js = js.replace(
    #     "df.to_json(self.output_file, orient='records', lines=True, force_ascii=False)", 
    #     "df.sample(100).to_json(self.output_file, orient='records', lines=True, force_ascii=False)" ## 采样10条而已。
    # )
    exec(js)

将全程使用CPU进行训练


/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
数据集大小: 10000
数据集样本示例: {'instruction': '这是一个尾实体预测的知识图谱任务，这是一个正例', 'input': '头实体为上海柜 资生堂男士焕能紧致凝霜50ml 保湿抗老化面霜，关系为关联场景', 'output': '尾实体为擦脸'}


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenized后的数据格式: ['input_ids', 'attention_mask', 'labels']
Input IDs示例: [105464, 101143, 101565, 104538, 107232, 28029, 101454, 88802, 3837, 105464]...
Labels示例: [105464, 101143, 101565, 104538, 107232, 28029, 101454, 88802, 3837, 105464]...


...

模型已移至CPU设备
[2025-10-08 17:39:10,800] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to mps (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2025-10-08 17:39:10,923] torch.distributed.elastic.multiprocessing.redirects: [WARNING] NOTE: Redirects are currently not supported in Windows or MacOs.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


使用CPU训练中，当前步数: 0
使用CPU训练中，当前步数: 0
使用CPU训练中，当前步数: 0
使用CPU训练中，当前步数: 0


Step,Training Loss
10,29.605000
20,12.346900
30,4.190400
40,2.421900
50,1.808200
60,1.339200
70,1.815300
80,1.759500
90,1.483600
100,1.373100


使用CPU训练中，当前步数: 50
使用CPU训练中，当前步数: 50
使用CPU训练中，当前步数: 50
使用CPU训练中，当前步数: 50
使用CPU训练中，当前步数: 100
使用CPU训练中，当前步数: 100
使用CPU训练中，当前步数: 100
使用CPU训练中，当前步数: 100
使用CPU训练中，当前步数: 150
使用CPU训练中，当前步数: 150
使用CPU训练中，当前步数: 150
使用CPU训练中，当前步数: 150
使用CPU训练中，当前步数: 200
使用CPU训练中，当前步数: 200
使用CPU训练中，当前步数: 200
使用CPU训练中，当前步数: 200
使用CPU训练中，当前步数: 250
使用CPU训练中，当前步数: 250
使用CPU训练中，当前步数: 250
使用CPU训练中，当前步数: 250
使用CPU训练中，当前步数: 300
使用CPU训练中，当前步数: 300
使用CPU训练中，当前步数: 300
使用CPU训练中，当前步数: 300
使用CPU训练中，当前步数: 350
使用CPU训练中，当前步数: 350
使用CPU训练中，当前步数: 350
使用CPU训练中，当前步数: 350
使用CPU训练中，当前步数: 400
使用CPU训练中，当前步数: 400
使用CPU训练中，当前步数: 400
使用CPU训练中，当前步数: 400
使用CPU训练中，当前步数: 450
使用CPU训练中，当前步数: 450
使用CPU训练中，当前步数: 450
使用CPU训练中，当前步数: 450
使用CPU训练中，当前步数: 500
使用CPU训练中，当前步数: 500
使用CPU训练中，当前步数: 500
使用CPU训练中，当前步数: 500
使用CPU训练中，当前步数: 550
使用CPU训练中，当前步数: 550
使用CPU训练中，当前步数: 550
使用CPU训练中，当前步数: 550
使用CPU训练中，当前步数: 600
使用CPU训练中，当前步数: 600
使用CPU训练中，当前步数: 600
使用CPU训练中，当前步数: 600
使用CPU训练中，当前步数: 650
使用CPU训练中，当前步数: 650
使用CPU训练中，当前步数: 650
使用CPU训练中，当前步数: 650
使用CPU训练中，当前步数: 7

# 推理测试

In [24]:
def get_text_embedding(text, model, tokenizer, device, max_length=256):
    """
    获取文本的embedding向量
    
    参数:
        text (str): 要获取embedding的文本
        model: 加载的模型
        tokenizer: 分词器
        device: 运行设备
        max_length: 最大序列长度
    
    返回:
        numpy.ndarray: 文本的embedding向量
    """
    # 分词处理
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding="max_length",
        return_tensors="pt"
    )
    
    # 将输入移至指定设备
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 设置模型为评估模式
    model.eval()
    
    # 禁用梯度计算以提高效率
    with torch.no_grad():
        # 获取模型输出
        outputs = model(**inputs, output_hidden_states=True)
        
        # 获取最后一层隐藏状态
        # 通常使用最后一层的平均池化作为文本的embedding
        last_hidden_state = outputs.hidden_states[-1]  # [batch_size, seq_length, hidden_size]
        
        # 计算所有token的平均值作为句子embedding
        # 可以根据需求选择不同的池化策略
        sentence_embedding = torch.mean(last_hidden_state, dim=1)  # [batch_size, hidden_size]
        
        # 如果需要，也可以使用[CLS]标记的输出作为embedding
        # cls_embedding = last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        
        # 处理BFloat16类型，先转换为float32再转为numpy
        if sentence_embedding.dtype == torch.bfloat16:
            sentence_embedding = sentence_embedding.to(torch.float32)
        
        # 转换为numpy数组并移至CPU
        sentence_embedding = sentence_embedding.cpu().numpy()
        
    return sentence_embedding[0]  # 返回单个样本的embedding
def load_model_for_embedding(model_path, use_quantization=True):
    """
    加载训练好的模型用于获取embedding
    
    参数:
        model_path (str): 模型路径
        use_quantization (bool): 是否使用量化
    
    返回:
        tuple: (model, tokenizer, device)
    """
    # 确定设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-1.8B", cache_dir=model_path)
    tokenizer.pad_token = tokenizer.eos_token

    # 使用CPU加载模型
    model_kwargs = {
        "cache_dir": model_path,
        "torch_dtype": torch.float32,  # CPU使用float32
        "device_map": None,  # 不使用device_map
        "trust_remote_code": True
    }
    base_model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen1.5-1.8B",
        **model_kwargs
    )
    
    # 加载lora权重
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, save_dir)
    
    # 移动模型到设备
    model.to(device)
    
    return model, tokenizer, device

In [25]:
sample_text = "这是一个测试文本，用于获取embedding。"

In [26]:
# 加载模型
print("正在加载模型...")
model, tokenizer, device = load_model_for_embedding(model_path)

正在加载模型...
使用设备: cpu


In [27]:
# 获取embedding
print("正在获取embedding...")
embedding = get_text_embedding(sample_text, model, tokenizer, device)

# 打印结果
print(f"获取的embedding形状: {embedding.shape}")
print(f"embedding示例值: {embedding[:10]}")

# 释放内存
del model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

正在获取embedding...
获取的embedding形状: (2048,)
embedding示例值: [-1.1167952   0.34650832 -0.2989049  -0.399235    0.18019523  0.71412265
 -0.2941041  -1.012376    0.9535338  -1.1594336 ]


In [ ]:
# model.print_trainable_parameters()